In [1]:

import heapq

import tensorflow as tf
from zoo import init_nncontext
from zoo.pipeline.api.net import TFOptimizer, TFDataset, TFPredictor
from bigdl.optim.optimizer import *
import numpy as np
import sys

from bigdl.dataset import mnist
from bigdl.dataset.transformer import *

sys.path.append("./slim")  # add the slim library
from nets import lenet

slim = tf.contrib.slim



In [2]:
max_epoch = 1
data_num = 60000
sc = init_nncontext()


# get data, pre-process and create TFDataset
def get_data_rdd(dataset):
    (images_data, labels_data) = mnist.read_data_sets("/tmp/mnist", dataset)
    image_rdd = sc.parallelize(images_data[:data_num])
    labels_rdd = sc.parallelize(labels_data[:data_num])
    rdd = image_rdd.zip(labels_rdd) \
        .map(lambda rec_tuple: [normalizer(rec_tuple[0], mnist.TRAIN_MEAN, mnist.TRAIN_STD),
                                np.array(rec_tuple[1])])
    return rdd

tf.reset_default_graph()


training_rdd = get_data_rdd("train")
testing_rdd = get_data_rdd("test")
dataset = TFDataset.from_rdd(training_rdd,
                             names=["features", "labels"],
                             shapes=[[28, 28, 1], []],
                             types=[tf.float32, tf.int32],
                             batch_size=1024,
                             val_rdd=testing_rdd
                             )

# construct the model from TFDataset
images, labels = dataset.tensors

with slim.arg_scope(lenet.lenet_arg_scope()):
    logits, end_points = lenet.lenet(images, num_classes=10, is_training=True)

loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=labels))


('Extracting', '/tmp/mnist/train-images-idx3-ubyte.gz')
('Extracting', '/tmp/mnist/train-labels-idx1-ubyte.gz')
('Extracting', '/tmp/mnist/t10k-images-idx3-ubyte.gz')
('Extracting', '/tmp/mnist/t10k-labels-idx1-ubyte.gz')
INFO:tensorflow:Scale of 0 disables regularizer.


In [3]:
%%time
# create a optimizer
optimizer = TFOptimizer(loss, Adam(1e-3),
                        val_outputs=[logits],
                        val_labels=[labels],
                        val_method=Top1Accuracy())
optimizer.set_train_summary(TrainSummary("/tmp/az_lenet", "lenet"))
optimizer.set_val_summary(ValidationSummary("/tmp/az_lenet", "lenet"))
# kick off training
optimizer.optimize(end_trigger=MaxEpoch(max_epoch))

saver = tf.train.Saver()
saver.save(optimizer.sess, "/tmp/lenet/")

creating: createAdam
creating: createTop1Accuracy
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
creating: createTFTrainingHelper
creating: createIdentityCriterion
creating: createMaxEpoch
creating: createDistriOptimizer
creating: createTFValidationMethod
creating: createEveryEpoch
creating: createTrainSummary
creating: createValidationSummary
creating: createMaxEpoch
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
TFDataset instance has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
TFDataset instance has no attribute 'name'
CPU times: user 2.52 s, sys: 201 ms, total: 2.73 s
Wall time: 2min 43s


In [5]:
# construct the model from TFDataset
tf.reset_default_graph()
dataset = TFDataset.from_rdd(testing_rdd,
                                 names=["features", "labels"],
                                 shapes=[[28, 28, 1], [1]],
                                 types=[tf.float32, tf.int32],
                                 batch_per_thread=20
                                 )
images, labels = dataset.tensors

labels = tf.squeeze(labels)

with slim.arg_scope(lenet.lenet_arg_scope()):
    logits, end_points = lenet.lenet(images, num_classes=10, is_training=False)

predictions = tf.to_int32(tf.argmax(logits, axis=1))
correct = tf.expand_dims(tf.to_int32(tf.equal(predictions, labels)), axis=1)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "/tmp/lenet/")

    predictor = TFPredictor(sess, [correct])

    accuracy = predictor.predict().mean()

    print("predict accuracy is %s" % accuracy)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /tmp/lenet/
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
creating: createTFNet


IllegalArgumentException: u"NodeDef mentions attr 'Truncate' not in Op<name=Cast; signature=x:SrcT -> y:DstT; attr=SrcT:type; attr=DstT:type>; NodeDef: ToInt32 = Cast[DstT=DT_INT32, SrcT=DT_INT64, Truncate=false](ArgMax). (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.)."

In [6]:
import tensorflow
tensorflow.__version__

'1.11.0'